In [25]:
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import hypertools as hyp
import time, glob, itertools
import geoplot as gplt

from scipy import spatial

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='ticks', context='talk')
%matplotlib inline

In [2]:
path = r'J:\\Geology\\WSGS\\Projects\\Critical Minerals\\probability maps\\'
allFiles=glob.glob(path+"*.csv")
frame=pd.DataFrame
listed=[]
for file in allFiles:
    df = pd.read_csv(file, index_col=[0])
    listed.append(df)
frame = pd.concat(listed)

In [3]:
huc12=gpd.read_file(r'J:\Geology\WSGS\GIS\huc_12.shp') #reads in the hydraulic basin units and adds an area column
huc12= huc12.to_crs({'init': 'epsg:3732'})


In [4]:
len(huc12.HUC_12.unique())

2382

In [5]:
frame['HUC_12']=frame.HUC_12.astype(str)

In [6]:
CaptainHucstable = huc12.merge(frame, on='HUC_12' )

In [62]:
inHouse=pd.read_csv(r'J:\Geology\WSGS\Projects\Critical Minerals\inhouse_elemental.csv')
inHouse

,Publication,Sample_ID,Old_Sample_ID,WyoDoG,WyoDoG_Name,Sample_Desc,latitude,longitude,ag_ppm,al_pct,...,ti_pct,ti_ppm,tm_ppm,u_ppm,v_ppm,w_ppm,y_ppm,yb_ppm,zn_ppm,zr_ppm
0,REE_RI-65,20110921WS-B,NaN,NaN,Union Pass - Warm Spring Area (Northwest Fremo...,NaN,43.4843,-109.8640,1.63,4.07,...,0.03,300.0,NaN,54.90,319.0,9.9,323.0,NaN,170.0,500.0
1,Iron_RI-67,20141015WS-D,NaN,X,Grass Creek north,Cretaceous Mesaverde Fm paleoplacer; Grass Cr...,43.9582,-108.6295,0.50,NaN,...,0.07,700.0,5.29,102.00,931.0,NaN,292.0,39.20,277.0,10000.0
2,REE_RI-71,WGS-0501,NaN,NaN,"Southeast Pine Mountain, Natrona County",NaN,42.9534,-106.9520,0.50,NaN,...,0.02,200.0,1.61,26.20,2500.0,5.0,102.5,12.55,1065.0,10000.0
3,REE_RI-71,20150929LC-2,NaN,NaN,Wilkins Peak Member,NaN,41.0059,-109.2228,1.30,NaN,...,0.12,1200.0,4.46,64.10,1200.0,8.0,182.0,32.20,470.0,10000.0
4,OtherSamples,20110921WS-B,NaN,NaN,South Fork of Warm Springs Creek - 2 heaping p...,South Fork of Warm Springs Creek - 2 heaping p...,43.4843,-109.8645,1.63,4.07,...,0.03,300.0,6.35,92.20,357.0,20.0,610.0,44.40,170.0,10000.0
5,REE_RI-65,WMS-GC1-2010,NaN,NaN,Mesaverde-Grass Creek South,NaN,43.9137,-108.6082,0.01,0.43,...,0.10,1000.0,6.93,135.00,768.0,5.6,331.0,50.90,306.0,10000.0
6,OtherSamples,K-PV1,NaN,NaN,"Kirwin area, Pickwick Vein","Kirwin area, Pickwick Vein",43.8605,-109.2948,154.00,0.49,...,0.09,900.0,0.02,8.46,26.0,19.0,0.5,0.09,141.0,2.0
7,REE_RI-65,20120730BG-2,NaN,NaN,Skunk Hollow,NaN,42.7384,-106.3162,0.50,NaN,...,3.40,34000.0,0.02,0.17,5.0,1.0,0.8,0.15,2.0,2.0
8,REE_RI-65,20120927JC-G.1,NaN,NaN,"Island City Group, Sierra Madre",NaN,41.2051,-107.0557,0.50,NaN,...,0.50,5000.0,0.89,0.13,97.0,1.0,56.0,6.18,34.0,2.0
9,REE_RI-65,20121009WS-F,NaN,NaN,20121009WS-F,NaN,42.6121,-107.7880,0.50,NaN,...,0.50,5000.0,0.05,6.47,46.0,3.0,3.3,0.24,429.0,2.0


In [8]:
from geopandas import GeoDataFrame
from shapely.geometry import Point
import fiona
#writes the point data to a shapefile in the dir called data.shp
geometry = [Point(xy) for xy in zip(inHouse.longitude, inHouse.latitude)]
crs = {'init': 'epsg:3732'} 
geo_df = GeoDataFrame(inHouse, crs={'init': 'epsg:4326'}, geometry=geometry)
#geo_df.to_file(driver='ESRI Shapefile', filename='data.shp')
projGeoDF=geo_df.to_crs(crs)

In [9]:
val_huc = gpd.sjoin(huc12, projGeoDF)

In [64]:
validations=pd.read_csv(r'J:\Geology\WSGS\Projects\Critical Minerals\validationmap.csv')

In [65]:
val_huc = val_huc.merge(validations, on='Sample_ID') #this is the inhouse data joined with the HUC dataset
val_huc

,AREA,PERIMETER,WY_HU12_,WY_HU12_ID,HUC_8,HUC_10,HUC_12,ACRES,STATES,NCONTRB_A,...,tm_ppm,u_ppm,v_ppm,w_ppm,y_ppm,yb_ppm,zn_ppm,zr_ppm,formation_x,formation_y
0,0.012441,0.634663,255,104,10080014,1008001404,100800140408,26978.198886,"MT,WY",0.0,...,0.11,1.72,23.0,1.0,6.6,0.70,23.0,173.0,Phosphoria Formation,Phosphoria Formation
1,0.018319,1.097871,268,140,10110201,1011020102,101102010204,39728.062947,WY,0.0,...,0.08,0.72,5.0,1.0,11.7,0.55,17.0,5.0,Belle Fourche Shale,Belle Fourche Shale
2,0.014678,0.704754,470,179,10080010,1008001002,100800100202,32021.898860,WY,0.0,...,0.05,0.57,14.0,1.0,3.5,0.49,7.0,91.0,Cloverly Formation,Cloverly Formation
3,0.013295,0.745599,545,206,10080010,1008001002,100800100205,29063.529586,WY,0.0,...,5.20,33.80,33.0,2.0,430.0,32.80,21.0,322.0,Morrison Formation,Morrison Formation
4,0.013295,0.745599,545,206,10080010,1008001002,100800100205,29063.529586,WY,0.0,...,0.77,5.21,62.0,2.0,51.3,5.07,36.0,230.0,Morrison Formation,Morrison Formation
5,0.013295,0.745599,545,206,10080010,1008001002,100800100205,29063.529586,WY,0.0,...,0.20,14.15,39.0,1.0,11.8,1.30,74.0,214.0,Morrison Formation,Morrison Formation
6,0.013295,0.745599,545,206,10080010,1008001002,100800100205,29063.529586,WY,0.0,...,0.15,0.86,20.0,1.0,8.3,1.21,10.0,303.0,Cloverly Formation,Cloverly Formation
7,0.013295,0.745599,545,206,10080010,1008001002,100800100205,29063.529586,WY,0.0,...,0.02,1.67,8.0,1.0,2.5,0.20,5.0,13.0,Cloverly Formation,Cloverly Formation
8,0.013295,0.745599,545,206,10080010,1008001002,100800100205,29063.529586,WY,0.0,...,0.28,4.76,16.0,1.0,21.6,2.01,3.0,4.0,Cloverly Formation,Cloverly Formation
9,0.013295,0.745599,545,206,10080010,1008001002,100800100205,29063.529586,WY,0.0,...,0.61,2.26,54.0,2.0,39.4,4.14,40.0,255.0,Cloverly Formation,Cloverly Formation


In [21]:
true_vals = []
for samples in tqdm(range(len(val_huc))):
    true_vals.append(val_huc.loc[samples].HUC_12)

bflat = []  
for samples in tqdm(range(len(val_huc))):
    formation = val_huc.loc[samples].formation[0:10]
    b = []
    for element in range(34,99):
        ppm = val_huc.columns[element] #select each element in the list
        valued = val_huc.loc[samples][element] #select each elements values 
        #from the big list, select element, formation, and concentration greater than the sample has, and create a list of the
        #basins that we should explore
        b0 = CaptainHucstable[(CaptainHucstable['element']== ppm) & (CaptainHucstable['formation'].str.contains(formation)) &(
            CaptainHucstable['bins_ppm']>= valued) & (CaptainHucstable['prob']>0.6)].HUC_12.values
        b.append(b0)
    bflat.append(np.unique(np.asarray([item for sublist in b for item in sublist])))
preds = np.unique(np.asarray([item for sublist in bflat for item in sublist]))
TP = len(set(true_vals).intersection(preds))
FP = len(preds)-len(set(true_vals).intersection(preds))
TN = len(huc12.HUC_12.unique())-len(preds)
FN = len(set(true_vals).intersection(set(huc12.HUC_12.unique()) ^ set(preds)))

precision = TP/(TP+FP)
recall = TP/(TP+FN)
acc = (TP+TN)/(TP+TN+FP+FN)

100%|█████████████████████████████████████████████████████████████████████████████| 101/101 [2:58:47<00:00, 106.21s/it]


In [22]:
print([precision, recall, acc ])

[0.025193798449612403, 0.8125, 0.36511919698870765]


In [23]:
F1 = 2*((precision*recall)/(precision+recall))
F1

0.04887218045112781

In [24]:
print([TP, FP, TN, FN])

[39, 1509, 834, 9]


In [ ]:
#df=pd.DataFrame(columns=val_huc.columns[34:-1])
#df['Sample_ID'] = val_huc.Sample_ID
#choices = []
Fone = []



for samples in tqdm(range(len(val_huc))):
    TP = []
    FP = []
    TN = []
    FN = []
    a = val_huc.loc[samples].HUC_12 #select the samples huc
    formation = val_huc.loc[samples].formation[0:10]
    b = []
    for element in range(34,99):
        ppm = val_huc.columns[element] #select each element in the list
        valued = val_huc.loc[samples][element] #select each elements values 
        #from the big list, select element, formation, and concentration greater than the sample has, and create a list of the
        #basins that we should explore
        b0 = CaptainHucstable[(CaptainHucstable['element']== ppm) & (CaptainHucstable['formation'].str.contains(formation)) &(
            CaptainHucstable['bins_ppm']>= valued) & (CaptainHucstable['prob']>0.1)].HUC_12.values
        b.append(b0)
    bflat = np.unique(np.asarray([item for sublist in b for item in sublist]))

    if a in bflat:
        #df[ppm].iloc[samples]=1 #if the actual HUC matches the list of predicted HUC's then we record a 1
        TP.append(1)
        FP.append(len(b)-1)
        TN.append(len(huc12.HUC_12.unique())-len(b))
        FN.append(0)
                       
    else: 
        TP.append(0)
        FP.append(len(b))
        TN.append(len(huc12.HUC_12.unique())-1)
        FN.append(1)
            #df[ppm].iloc[samples]=0 #if they don't match we record a 0
        #choices.append(len(b))
    precision = np.nan_to_num(np.asarray(TP)/(np.asarray(TP)+np.asarray(FP)))
    recall = np.asarray(TP)/(np.asarray(TP)+np.asarray(FN))
    acc = (np.asarray(TP)+np.asarray(TN))/(np.asarray(TP)+np.asarray(TN)+np.asarray(FP)+np.asarray(FN))
    F1 = np.nan_to_num(2*((precision*recall)/(precision+recall)))
    Fone.append(F1)
#df.to_csv(r'J:\Geology\WSGS\Projects\Critical Minerals\accuracy_60percent_probability.csv')


In [24]:
inHouse.columns

Index(['Publication', 'Sample_ID', 'Old_Sample_ID', 'WyoDoG', 'WyoDoG_Name',
       'Sample_Desc', 'latitude', 'longitude', 'ag_ppm', 'al_pct', 'as_ppm',
       'au_ppm', 'ba_ppm', 'be_ppm', 'bi_ppm', 'ca_pct', 'cd_ppm', 'cd_ppm.1',
       'co_ppm', 'cr_ppm', 'cs_ppm', 'cu_ppm', 'dy_ppm', 'er_ppm', 'eu_ppm',
       'fe_pct', 'ga_ppm', 'gd_ppm', 'ge_ppm', 'hf_ppm', 'hg_ppm', 'ho_ppm',
       'in_ppm', 'k_pct', 'la_ppm', 'li_ppm', 'lu_ppm', 'mg_pct', 'mn_ppm',
       'mo_ppm', 'na_pct', 'nb_ppm', 'nd_ppm', 'ni_ppm', 'p_ppm', 'pb_ppm',
       'pd_ppm', 'pr_ppm', 'pt_ppm', 'rb_ppm', 're_ppm', 's_pct', 'sb_ppm',
       'sc_ppm', 'se_ppm', 'sm_ppm', 'sn_ppm', 'sr_ppm', 'ta_ppm', 'tb_ppm',
       'te_ppm', 'th_ppm', 'ti_ppm1', 'ti_pct', 'ti_ppm', 'tm_ppm', 'u_ppm',
       'v_ppm', 'w_ppm', 'y_ppm', 'yb_ppm', 'zn_ppm', 'zr_ppm', 'geometry'],
      dtype='object')

In [66]:
path = r'J:\\Geology\\WSGS\\GIS\\Predicted locations\\'
allFiles=glob.glob(path+"*.shp")
points = projGeoDF


validations = []
for shapefile in allFiles:
    shape = gpd.read_file(shapefile)
    shape = shape.to_crs({'init': 'epsg:3732'})
    chosen = gpd.sjoin(points, shape)
    chosen['element'] = shapefile[-10:-4]
    validations.append(chosen)

In [78]:
selectedSamples = pd.concat(validations)

In [83]:
predictions = []
actual = []
for i in range(selectedSamples.shape[0]):
    predictions.append(selectedSamples.iloc[i].bins_ppm)
    actual.append(selectedSamples.iloc[i][selectedSamples.iloc[0].element])
    

In [97]:
from sklearn.metrics import mean_squared_error


array([5.00e-01, 5.00e-01, 5.00e-01, 5.00e-01, 5.00e-01, 5.00e-01,
       5.00e-01, 5.00e-01, 5.00e-01, 5.00e-01, 7.00e-01, 5.00e-01,
       5.00e-01, 1.30e-01, 5.20e-01, 5.00e-01, 5.00e-01, 5.00e-01,
       1.30e-01, 5.00e-01, 5.00e-01,      nan, 1.00e+00, 1.00e+00,
            nan,      nan, 1.00e+00, 5.00e-01, 5.00e-01, 5.00e-01,
       1.00e+00, 5.00e-01, 5.00e-01, 5.00e-01, 5.00e-01, 5.00e-01,
       5.00e-01, 5.00e-01, 5.00e-01, 2.20e+00, 5.00e-01, 5.00e-01,
       5.00e-01, 1.00e-02, 5.00e-01, 5.00e-01,      nan, 3.00e-02,
       5.00e-01, 5.00e-01, 5.00e-01, 5.00e-01, 5.00e-01, 5.00e-01,
       5.00e-01, 7.00e-01, 5.00e-01, 5.00e-01, 5.00e-01, 5.00e-01,
       2.10e-01, 1.87e+00, 1.36e+02, 5.00e-01, 5.00e-01,      nan,
       1.00e+00, 1.00e+00,      nan,      nan, 5.00e-01, 5.00e-01,
       5.00e-01, 5.00e-01, 5.00e-01, 5.00e-01, 5.00e-01, 5.00e-01,
       2.20e+00, 5.00e-01, 5.00e-01, 5.00e-02, 1.00e-02, 1.00e+00,
       5.00e-01, 2.00e-02, 3.00e-02, 1.50e+00, 5.00e-01, 5.00e

In [99]:
mean_squared_error(np.nan_to_num(np.asarray(actual)), np.asarray(predictions))

19272235.744467396